For background information, refer to the ReadMe on github
* https://github.com/wpbSabi/sustainable_supply_chain_template 

In [7]:
# !pip install -U googlemaps 
import folium
import numpy as np
import pandas as pd
import googlemaps

# Data

## Geocoding

My first choice would have been to use geopy / nominatum to obtain latitudes and longitudes based on city names. However, there was an issue on April 8, 2023 with one of the services geopy relies on.  The good news is that on April 18, 2023 the service was running again.
* https://geopy.readthedocs.io/en/stable/

In [8]:
# from geopy.geocoders import Nominatim # OpenStreetMap API for geocoding
# geolocator = Nominatim(user_agent="april_8")
# city = 'Portland, Oregon'
# df = pd.DataFrame(list_of_cities, columns=['City'])
# for c in df['City']:
#     location = geolocator.geocode(city)
#     df.loc[c, 'latitude'] = location.latitude
#     df.loc[c, 'longitude'] = location.longitude
# df = df[df.isna()['City']==True]\
#     .reset_index()\
#     .rename(columns={'index': 'location'})\
#     .drop('City', axis=1)
# df

Instead, I signed up for a google maps api

In [9]:
# gmaps_key = googlemaps.Client(key='<YOUR_KEY_HERE')

# # Import or set format for data
# df = pd.read_csv('data/location_lat_long.csv')
# # Create a list, to retrieve multiple locations at a time
# list_of_cities = ['Salt Lake City, Utah',
#                   'New Orleans, Louisiana',
#                   'Hartford, Connecticut',
#                   'Buffalo, New York',
#                   'Birmingham, Alabama']
# for city in list_of_cities:
#     g = gmaps_key.geocode(city)
#     address = g[0]['formatted_address']
#     lat = g[0]['geometry']['location']['lat']
#     long = g[0]['geometry']['location']['lng']
#     df.loc[len(df), 'location'] = address
#     df.loc[len(df) - 1, 'latitude'] = lat
#     df.loc[len(df) - 1, 'longitude'] = long

In [10]:
# def map_locations(df):
#     # Verify the new data by creating a map (or look at the df)
#     m = folium.Map([40, -95],  zoom_start=4)
#     # Add orange circles
#     for i in range(len(df)): folium.CircleMarker(
#         location=[df.iloc[i]['latitude'],
#                 df.iloc[i]['longitude']],
#         tooltip=df.iloc[i]['location'],
#         color='orange',
#         fill=True,
#         fill_opacity=0.7,
#         radius=5 # df.iloc[i]['demand_fix'] #5 #df.iloc[i]['Demand FY21']
#         ).add_to(m)
#     # To enable the toggle between map base layers
#     folium.TileLayer('OpenStreetMap').add_to(m)
#     folium.TileLayer('Stamen Terrain').add_to(m)
#     folium.TileLayer('Stamen Toner').add_to(m)
#     folium.TileLayer('Stamen Water Color').add_to(m)
#     folium.TileLayer('CartoDB positron').add_to(m)
#     folium.TileLayer('CartoDB dark_matter').add_to(m)
#     folium.LayerControl().add_to(m)
#     # m.save('images/python_folium_bubble_map.html')
#     return m
# map_locations(lat_long_data)

In [11]:
# Overwrite the data file with the updates
# df.to_csv('data/location_lat_long.csv', index=False)

## Greatest Circle Distance (GCD)

In [12]:
# Import the geodata from a file. More geodata may be added in the next steps.
lat_long_data = pd.read_csv('data/location_lat_long.csv')

# Create a data frame of the possible Origin / Destination pairs
origins = lat_long_data.copy()
destinations = lat_long_data.copy()
origin_destination_pairs = origins.merge(destinations, how='cross')

In [13]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the greatest circle distance between two points on Earth by 
        utilizing the haversine calculation via latitude and longitude data.
    """
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 \
        + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 2)

In [14]:
# Distance in miles from Portland to Boise, used in the greenhouse gas example
print(lat_long_data.loc[0])
print(haversine_distance(lat_long_data.loc[0, 'latitude'],
                   lat_long_data.loc[0, 'longitude'],
                   43.6150,
                   -116.2023))

location     Portland, OR, USA
latitude             45.515232
longitude          -122.678385
Name: 0, dtype: object
554.64


In [15]:
def calculate_distances(df):
    """
    Utilize haversine_distance() and the df for origin / destination pairs
    Calculate the distances between all origin / destination pairs
    """
    df['distance'] = df\
        .apply(lambda x: haversine_distance(x['latitude_x'],
                                        x['longitude_x'],
                                        x['latitude_y'],
                                        x['longitude_y']), axis=1)
    # For shipments within a city, set the distance to 10 miles
    df['distance'] = np.where(df['location_x']==df['location_y'],
                              10,
                              df['distance'])
    return df 
distance_between_od_pairs = calculate_distances(origin_destination_pairs)

## Scenarios / Greenhouse gas calculations
Create shipments for scenarios, and assumptions:
* 0.36 grams co2e per mile for parcels shipments or LTL shipments
* DC footprint - 443,000 grams co2e per day

### One Distribution Center

In [16]:
def footprint_for_one_dc(df):
    """
    Return the emissions from one daily shipment to each of the 50 cities
    Returns total emissions and transport emissions in kg co2e
    """
    co2e_per_mile_per_truck = 0.36 # grams of co2e per mile for LTL truck
    co2e_for_facility = 443000 # grams of co2e, average distribution center
    df = df.assign(co2e_shipment = df['distance'] * co2e_per_mile_per_truck)
    total_kg_co2e = round(sum(df['co2e_shipment']) + co2e_for_facility)
    transport_kg_co2e = round(sum(df['co2e_shipment']) )
    return round(total_kg_co2e / 1000), round(transport_kg_co2e / 1000)

In [19]:
def map_supply_chain_network(df):
    """
    Creates a map of origins and destinations
    df contains the destinations
    dc_list contains the distribution centers
    """
    # Verify the new data by creating a map (or look at the df)
    m = folium.Map([40, -95],  zoom_start=4)
    # Add darkgreen circles for destinations
    for j in range(len(df)): 
        folium.CircleMarker(
            location=[df.iloc[j]['latitude_y'],
                    df.iloc[j]['longitude_y']],
            tooltip=df.iloc[j]['location_y'],
            color='darkgreen',
            fill=True,
            fill_opacity=0.7,
            radius=5 
        ).add_to(m)
    for i in df['location_x'].unique(): 
        dc = df[df['location_x']==i].reset_index(drop=True)
        # Add lines to show Origin-Destination pairs
        add_lines_df = pd.DataFrame()
        for k in range(0, len(df)):
            od = [(df['latitude_x'][k], df['longitude_x'][k]),
                 (df['latitude_y'][k], df['longitude_y'][k])]
            add_lines_df[k]=od
            folium.PolyLine(add_lines_df[k], color='darkgreen').add_to(m)
        # Add blue circles for distribution centers
        folium.CircleMarker(
            location=[dc.iloc[0]['latitude_x'],
                    dc.iloc[0]['longitude_x']],
            tooltip=dc.iloc[0]['location_x'],
            color='blue',
            fill=True,
            fill_opacity=0.3,
            radius=10
        ).add_to(m)    
    # To enable the toggle between map base layers
    folium.TileLayer('OpenStreetMap').add_to(m)
    folium.TileLayer('CartoDB positron').add_to(m)
    folium.TileLayer('CartoDB dark_matter').add_to(m)
    folium.LayerControl().add_to(m)
    return m

In [20]:
# Scenario 1: Distribution Center at Memphis
scen1 = distance_between_od_pairs[
    distance_between_od_pairs['location_x']=='Memphis, TN, USA'
].reset_index(drop=True)
display(footprint_for_one_dc(scen1))
map_supply_chain_network(scen1)

(467, 24)

In [21]:
# Scenario 2: Distribution Center at St Louis
scen2 = distance_between_od_pairs[
    distance_between_od_pairs['location_x']=='St. Louis, MO, USA'
].reset_index(drop=True)
display(footprint_for_one_dc(scen2))
map_supply_chain_network(scen2)

(466, 23)

In [14]:
# Scenario 3: Distribution Center at Seattle (probably the worst choice)
scen3 = distance_between_od_pairs[
    distance_between_od_pairs['location_x']=='Seattle, WA, USA'
].reset_index(drop=True)
display(footprint_for_one_dc(scen3))
map_supply_chain_network(scen3)

(494, 51)

In [15]:
print((494 - 466) / 494)
print((494 - 466) / 466)

0.05668016194331984
0.060085836909871244


### Three Distribution Centers

In [40]:
# Scenario 4: Riverside, Houston, and Columbus
s4 = distance_between_od_pairs[
    (distance_between_od_pairs['location_x']=='Riverside, CA, USA') | 
    (distance_between_od_pairs['location_x']=='Houston, TX, USA') | 
    (distance_between_od_pairs['location_x']=='Columbus, OH, USA')
]

In [41]:
# For each destination, identify the closest facility
scen4 = s4[['location_y', 'distance']]\
            .groupby(['location_y'], as_index=False)\
            .min()
            
# Filter on the data frame by the closest facility
scen4_all = s4[s4['distance'].isin(scen4['distance'])].reset_index(drop=True)
scen4_all.tail()

,location_x,latitude_x,longitude_x,location_y,latitude_y,longitude_y,distance
45,"Columbus, OH, USA",39.961176,-82.998794,"Richmond, VA, USA",37.540725,-77.436048,552.24
46,"Columbus, OH, USA",39.961176,-82.998794,"Louisville, KY, USA",38.252665,-85.758456,304.59
47,"Columbus, OH, USA",39.961176,-82.998794,"Hartford, CT, USA",41.765804,-72.673372,890.58
48,"Columbus, OH, USA",39.961176,-82.998794,"Buffalo, NY, USA",42.886447,-78.878369,473.00
49,"Columbus, OH, USA",39.961176,-82.998794,"Birmingham, AL, USA",33.518589,-86.810357,792.60


In [64]:
def footprint_for_multiple_dc(df):
    """
    Return the emissions from one daily shipment to each of the 50 cities
    Single-source each destination with only one DC
    Use the supply chain square root rule of thumb to scale DCs
    Returns total emissions and transport emissions in kg co2e
    """
    # Count the number of DCs
    number_of_dcs = len(df['location_x'].unique())
    # Add the greenhouse gas factors per transit mile and per facility / DC
    co2e_per_mile_per_truck = 0.36 # grams of co2e for LTL truck
    # grams of co2e, average distribution center scaled to number of dcs
    co2e_per_facility_daily = round(443000 / np.sqrt(number_of_dcs))
#     print('Average facility size:', co2e_per_facility_daily)
    
    # Calculate co2e based on distance
    df = df.assign(co2e_shipment = df['distance'] * co2e_per_mile_per_truck)
    
    # Total kg co2e 
    total_kg_co2e = round(
        sum(df['co2e_shipment']) + (co2e_per_facility_daily * number_of_dcs))
    
    # kg co2e from transportation
    transport_kg_co2e = round(sum(df['co2e_shipment']))

    return round(total_kg_co2e / 1000), round(transport_kg_co2e / 1000)

In [65]:
footprint_for_multiple_dc(scen4_all)

(779, 11)

In [66]:
print((779 - 466) / 779)

0.4017971758664955


In [67]:
map_supply_chain_network(scen4_all)

# Possible supply chain objective functions

- **Minimize the distance and minimize the cost**. Wrap one of the objectives into the constraint.


\begin{equation}
\text{Min} \quad Cost = \sum_{(d,c) \in \text{D} \times \text{C}}({\text{cost_per_mile}_{d,c}} * {\text{chosen_route}_{d,c}})
\end{equation}

\begin{equation}
\ {\text{chosen_route}_{d,c}} * {\text{distance}_{d,c}} \leq 500 \quad  
\end{equation}

\begin{equation}
\forall c \in \text{C, } d \in \text{D}, \text{chosen_route} \in {0, 1}
\end{equation}

- **Minimize the distance and minimize the cost**. Weight each objective.

\begin{equation}
\text{Min} \quad Cost = \sum_{(d,c) \in \text{D} \times \text{C}}(\text{A} * {\text{cost_per_mile}_{d,c}} * {\text{chosen_route}_{d,c}}) + (\text{B} * {\text{distance}_{d,c}} * {\text{chosen_route}_{d,c}})
\end{equation}

\begin{equation}
\forall c \in \text{C, } d \in \text{D}, \text{chosen_route} \in {0, 1}
\end{equation}

- **Minimize the distance and minimize the cost**. Look at cost first and then distance


\begin{equation}
\text{Min} \quad Cost = \sum_{(d,c) \in \text{D} \times \text{C}}({\text{cost_per_mile}_{d,c}} * {\text{chosen_route}_{d,c}})
\end{equation}

\begin{equation}
\ {\text{cost_per_mile}_{d,c}} * {\text{chosen_route}_{d,c}} \leq $1000 \quad  
\end{equation}

\begin{equation}
\forall c \in \text{C, } d \in \text{D}, \text{chosen_route} \in {0, 1}
\end{equation}

\begin{equation}
\text{Min} \quad Cost = \sum_{(d,c) \in \text{D} \times \text{C}}({\text{distance}_{d,c}} * {\text{chosen_route}_{d,c}})
\end{equation}

\begin{equation}
\ {\text{distance}_{d,c}} * {\text{chosen_route}_{d,c}} \leq 500 \quad  
\end{equation}

\begin{equation}
\forall c \in \text{C, } d \in \text{D}, \text{chosen_route} \in {0, 1}
\end{equation}

- **Minimize the planned cost**.  Planned cost includes the fixed cost of setup and the variable cost of shipping products from the Distribution Center to Customers.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(d,c) \in \text{Distribution Centers} \times \text{Customers}}({\text{shipping_cost}_{d,c}} * {\text{allocation}_{d,c}}) +  ({\text{fixed_cost}_{d}} * {\text{open_DC}_{d}})
\end{equation}

### Constraints

- **Fulfill Customer Demand**: Customers to receive allocation from the distribution centers in the amount of the planned demand.

\begin{equation}
\sum_{d \in \text{Distribution Centers}}{\text{allocation}_{d,c}} \geq \text{Demand}_{c} \quad  \forall c \in \text{Customers}
\end{equation}

- **Only allocate products to Open Distribution Centers**: A modeling constraint to ensure closed distribution centers do not ship products.  The big M is set to the sum of all customer demand.

\begin{equation}
\ {\text{allocation}_{d,c}} \leq M * \text{open_DC}_{d} \quad  \forall c \in \text{Customers, } d \in \text{Distributions}
\end{equation}

- **Distribution Centers Capacity**: Each Distribution Center has a limited capacity based on these planning decisions.

\begin{equation}
\sum_{c \in \text{Customers}}{\text{allocation}_{d,c}} \leq \text{Capacity}_{d} \quad \forall d \in \text{Distribution Centers}
\end{equation}
 